In [1]:
include("../code/julia/ha-trade.jl")

using MINPACK
using Plots
using CSV
using DataFrames

### Example with tariffs and transfers

As the title says!


In [2]:
γ = 1.5 # curvatuve on CRRA utility function
σϵ = 0.25 # logit dispersion parameter
Ncntry = 2 # number of countries

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ);

In [3]:
# here are some simpe country parametrers
TFP = [1.0; 1.0]

L = [1.0; 1.0]

d_ij = 2.05

d = [1.0 d_ij; d_ij 1.0]

tariff = [0.0 0.25; 0.0 0.0] # the tariff rate matrix (defualt is zero)
# here country 1 is imposing a 25 percent tariff on country 2

# this sets up the country specific paramters
cntry_prm = country_params(Ncntry = Ncntry, L = L, d = d, TFP = TFP, tariff = tariff);

In [5]:
f(x) = world_equillibrium_FG_τ((x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the tariff case (with finacial globalization)...
# so this means that what is taking place is that the government choses a transfer τ
# so that the reveue collected equalls the lump sum transfer

function f!(fvec, x)

    fvec .= f(x)

end

τvec = [0.01; 0.0]

xguess = [1.0;  τvec; 1.0147]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, (xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

print(sol)

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     4.828222e-02     0.000000e+00         0.141000
     2     1.153011e-03     9.307746e-03         0.796000
     3     2.040069e-05     5.729792e-06         0.188000
     4     6.701057e-07     3.003648e-10         0.156000
     5     8.097669e-09     4.189037e-13         0.173000
     6     7.035443e-11     4.215146e-19         0.148000
     7     6.052093e-12     1.093386e-20         0.176000
     8     1.915135e-15     6.847966e-23         0.173000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [1.0, 0.01, 0.0, 1.0147]
 * Zero: [1.0986598188185568, 0.01496580510882659, 8.309640152570302e-28, 1.0133160605657816]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 1.951000 seconds
 * Function Calls: 8
 * Jacobian 

In [7]:
wage, τ, R = unpack_xvec(sol.x, 2)

Y, tradeflows, A_demand, Gbudget, tradeshare, hh, dist  = world_equillibrium(R, wage, τ, hh_prm, cntry_prm);

Then here is quick overview of what is comming out of this

In [12]:
println(τ)

println(wage)

println(R)

[0.01496580510882659, 8.309640152570302e-28]
[1.0986598188185568, 1.0]
[1.0133160605657816, 1.0133160605657816]


This is the transfer back to household and also notice that this the tariff increased the demand for home goods leading to a larger relative wage rate and a lower rate of interest (see notebook [here](two-country.ipynb)). 

In [8]:
println(Gbudget)

[-1.5612511283791264e-17, -8.309640152570302e-28]


So the government budget constraint is holding

In [16]:
tradeshare

2×2 Matrix{Float64}:
 0.932819   0.0671806
 0.0746229  0.925377

Then this is the trade share matrix where you can see relative to the [no tariff case](two-country.ipynb), trade declined in both economies. 